# Part 3.2. Visdom

In [29]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

## 1. Import
`python -m visdom.server`로 서버를 키면 **localhost:8097**로 서버가 열린다. 

In [4]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


## 2. Usage

In [5]:
# text
vis.text("Hello World!", env="main")

'window_38482487aedaf8'

In [6]:
# image
a = torch.randn(3, 200, 200)
vis.image(a)

'window_384824a505234a'

In [7]:
# images
vis.images(torch.Tensor(3, 3, 28, 28))

'window_384824b16196f4'

## 3. MNIST and CIFAR Example

In [8]:
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)


Extracting ./MNIST_data\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_data\MNIST\raw



Extracting ./MNIST_data\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_data\MNIST\raw



Extracting ./MNIST_data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_data\MNIST\raw



Extracting ./MNIST_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_data\MNIST\raw
Processing...
Done!


Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./cifar10\cifar-10-python.tar.gz


Extracting ./cifar10\cifar-10-python.tar.gz to ./cifar10


In [9]:
# CIFAR
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_3848255d17376a'

In [10]:
# MNIST
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_3848256a94a8ce'

In [11]:
# dataloader
data_loader = torch.utils.data.DataLoader(dataset = MNIST, batch_size = 32, shuffle = False)
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break
vis.close(env="main")

torch.Size([32, 1, 28, 28])


''

## 4. Line Plot

In [17]:
Y_data = torch.randn(5)
plt = vis.line (Y=Y_data)    # X값이 없으면 0-1사이 값으로 자동 설정

In [18]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### 4.1. Line Update

In [19]:
# line update
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_384826482e8834'

### 4.2. Multiple Line on Single Windows

In [20]:
num = torch.Tensor(list(range(0,10)))  # (10, )
num = num.view(-1,1)                   # (10, 1)
num = torch.cat((num,num),dim=1)       # (10, 2)

# 선을 두 개 그리려면 X, Y의 형태가 같아야 한다
plt = vis.line(Y=torch.randn(10,2), X = num)

### 4.3. Line Info

In [21]:
# title 그래프 이름 / showlegend 선의 이름
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [22]:
# legend 선에 이름 직접 지정
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'],showlegend=True))

In [23]:
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

### 4.4. Draw Update Line

In [24]:
def loss_tracker(loss_plot, loss_value, num):
    # num과 loss_value는 Tensor
    vis.line(X=num, Y=loss_value, win = loss_plot, update='append')

In [25]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

## 5. MNIST CNN Example

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [27]:
# 하이퍼파라미터
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [30]:
# 데이터셋
mnist_train = dsets.MNIST(root='MNIST_data/', train = True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

In [31]:
# 데이터로더
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size = batch_size, shuffle =True, drop_last=True)

In [32]:
# 모델 설계

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [34]:
model = CNN().to(device)

# 임의의 값을 집어넣어서 모델 확인
value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


In [35]:
# cost function과 optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [36]:
# cost function plot
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [37]:
# 모델 학습
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

print('Learning Finished!')

KeyboardInterrupt: 

In [ ]:
# 모델 검증
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())